In [41]:
import autogen
config_list = [{'model': 'gpt-4', 'api_key': "sk-..."},{'model': 'gpt-3.5-turbo', 'api_key': "sk-..."},{'model': 'gpt-3.5-turbo-16k', 'api_key': "sk-.."}]

In [42]:
llm_config = {
    "seed": 65,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="user_proxy",
   max_consecutive_auto_reply=10,
   is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
   code_execution_config={"work_dir": "groupchat","use_docker": False},
   llm_config=llm_config,
   human_input_mode="TERMINATE",
   system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)
coder = autogen.AssistantAgent(
    name="Coder",
    system_message="You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor. Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor. If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.",
    llm_config=llm_config,
)
data_analyst = autogen.AssistantAgent(
    name="data_analyst",
    system_message="Expert in plot professional graphs using matplotlib python framework.",
    llm_config=llm_config,
)
web_developer = autogen.AssistantAgent(
    name="web_developer",
    system_message="Expert in build professional web apps using streamlit python framework. Always save the webapp file in the groupchat folder.",
    llm_config=llm_config,
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin, until admin approval.
The plan may involve a coder who can write code to solve tasks, a data analyst that present the results in a professional and visual way, and the web_developer that build professional web apps.
Explain the plan first. Be clear which step is performed by a coder,which step is performed by a data_analyst and which step is performed by the web_developer.
''',
    llm_config=llm_config,
)
executor = autogen.UserProxyAgent(
   name="executor",
   system_message="Save and Execute code produced by the coder in the groupchat folder.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat","use_docker": False},
   human_input_mode="NEVER"
)
groupchat = autogen.GroupChat(agents=[user_proxy,coder, data_analyst,web_developer, planner], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(manager, message="I want a web app that give me the dollar/real rate of the last month using yfinance and plot a graph with this result.")